In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots

In [59]:
PATH_HEAD = r'C:\Users\猛\Desktop\yoshiki_study\normalizing_flow\result\210318_232820'
coef_all = pd.read_csv(PATH_HEAD+'/coef_all.csv').set_index('Date')
target_names = ['UST', 'S&P500', 'TOPIX', 'JGB']
rename_dict = dict(zip(['Treasury_JPY', 'SPX_JPY', 'TPXDDVD Index', 'SPJGBTR Index'], target_names))

df_org = pd.read_csv(PATH_HEAD+'/df_org.csv').set_index('Date').rename(columns=rename_dict)

In [63]:
df_coef = coef_all[coef_all.mean().drop_duplicates().index].iloc[:,1:]
index_test = df_coef.index

In [62]:
import itertools
corr_names = [i +'_' + j for i, j in itertools.product(target_names, target_names)]

In [190]:
roll_window = 50
df_corr_hist = pd.DataFrame(df_org.rolling(roll_window).corr().dropna().shift(-(roll_window-1)*df_org.shape[1]).loc[index_test].values.reshape(-1, df_org.shape[1]**2),
                            columns = corr_names, index = index_test)
df_corr_hist_back = pd.DataFrame(df_org.rolling(roll_window).corr().dropna().loc[index_test].values.reshape(-1, df_org.shape[1]**2),
                            columns = corr_names, index = index_test)

In [146]:
df_corr_pred = coef_all.rename(columns=dict(zip(coef_all.columns, df_corr_hist.columns)))
corr_pred = df_corr_pred[df_corr_pred.mean().drop_duplicates().index].iloc[:,1:]
extracted_names = corr_pred.columns
corr_hist = df_corr_hist[extracted_names]

In [188]:
import numpy as np
corr_corr = [np.corrcoef(corr_pred[i].values, corr_hist[i].values)[1, 0] for i in corr_pred.columns]
df_pred_hist = pd.DataFrame([comp_all.corr().iloc[i, i+int(comp_all.shape[1]/2)] for i in range(int(comp_all.shape[1]/2))],
                             index=extracted_names, columns=['corr_pred_hist'])

In [189]:
df_pred_hist

,corr_pred_hist
UST_S&P500,-0.344301
UST_TOPIX,-0.007012
UST_JGB,-0.152920
S&P500_TOPIX,0.155159
S&P500_JGB,-0.239717
TOPIX_UST,-0.007012
TOPIX_JGB,-0.339146
JGB_UST,-0.152920


In [144]:
fig = px.line(comp_all)
fig.show()